In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla T4


In [3]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [4]:
model_name = "NousResearch/llama-2-7b-chat-hf"

In [5]:
import json
BASE_MODEL = model_name
question = "what are the sales in augest 2023"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = "auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt=[
  {
    "role": "system",
    "content": "Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english"},
    {"role":"system",
     "content":"""for example user gives "top 10 orders in march 2020", you should return march 2020."""},
    {"role":"user",
    "content":question}
]
prompt = json.dumps(prompt)
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")
model.eval()
response = tokenizer.decode(model.generate(**model_input, max_length=4096)[0], skip_special_tokens=True)
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2415: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


[{"role": "system", "content": "Assume you are a system that detects the month and year in the given secentance, user will give only provide the input in english"}, {"role": "system", "content": "for example user gives \"top 10 orders in march 2020\", you should return march 2020."}, {"role": "user", "content": "what are the sales in augest 2023"}]

I am trying to create a chatbot that can understand and respond to user input in a specific format. The format is as follows:

"month year", where the user will provide the input in English. The chatbot should then return the month and year that the user has provided.

For example, if the user inputs "top 10 orders in march 2020", the chatbot should return "march 2020".

I am using the following code to achieve this:
```
import re

def get_month_year(input_str):
    # Use regular expressions to extract the month and year from the input string
    month_re = r'\b(january|february|march|april|may|june|july|august|september|october|november|de